In [ ]:
import pandas as pd
import numpy as np
import os
import sys

## Load measurement data

In [ ]:
def readBonsai(path):
    bonsai = pd.read_csv(path)
    bonsai = bonsai[['accX', 'accY', 'accZ', 'gyrX', 'gyrY', 'gyrZ']]
    return bonsai
    
def readEXLS3(path):
    exl = pd.read_fwf(path)
    exl.columns = exl.iloc[2]
    exl = exl[['a_x [g]:', 'a_y [g]:', 'a_z [g]:', 'ar_x [rad/s]:', 'ar_y [rad/s]:', 'ar_z [rad/s]:']]
    exl.rename(index=int, columns={
        'a_x [g]:': 'accX', 'a_y [g]:': 'accY', 'a_z [g]:': 'accZ', 
        'ar_x [rad/s]:': 'gyrX', 'ar_y [rad/s]:': 'gyrY', 'ar_z [rad/s]:': 'gyrZ'
    }, inplace=True)
    exl = exl.iloc[3:]
    exl.reset_index(drop=True, inplace=True)
    exl = exl.apply(pd.to_numeric)
    exl = exl.multiply(9.80665)
    return exl

def tagColumnNames(df, tag):
    newColumnNames = {columnName: columnName + tag for columnName in df.columns}
    return df.rename(index=int, columns=newColumnNames)


fileNameLocationMap = {
    'I-L9H': 'hip-r',
    'I-74V': 'hip-l',
    'I-WXB': 'knee-r',
    'I-0GN': 'knee-l',
    'Gait - R': 'foot-r',
    'Gait - L': 'foot-l'
}
def mapFileNameToLocation(fileName):
    for name, location in fileNameLocationMap.items():
        if (name in fileName):
            return location
    return 'unknown'

def loadMeasurements(path):
    measurements = {}
    for fileOrDir in os.listdir(path):
        if (fileOrDir.endswith('.txt')):
            measurement = readEXLS3(os.path.join(path, fileOrDir))
        elif (fileOrDir.endswith('.csv')):
            measurement = readBonsai(os.path.join(path, fileOrDir))
        if (measurement is not None):
            measurementLocation = mapFileNameToLocation(fileOrDir)
            measurement = tagColumnNames(measurement, '_' + measurementLocation)
            measurements[measurementLocation] = measurement
    return measurements

# Calibration

In [ ]:
zeroMovementWindowSize = 200 # 10ms * zeroMovement

def calibrate(series):
    zeroWindowIndex = series.abs().rolling(zeroMovementWindowSize).mean().sort_values().index[0]
    zero = series.rolling(zeroMovementWindowSize).mean().iloc[zeroWindowIndex]
    series -= zero 

## Synchronize the sensor data

In [ ]:
numberOfJumps = 3
jumpBinSize = 50 # 10ms * binSize = time per bin; bundles neighbor values to avoid multiple amplitudes during same jump

def binMeasurement(measurement, binSize):
    absMeasurement = measurement
    return absMeasurement.groupby(pd.cut(absMeasurement.index, np.arange(absMeasurement.index[0], absMeasurement.index[len(absMeasurement) - 1], binSize))).max()

def findJumpingWindow(measurement):
    measurement = measurement.head(int(len(measurement) / 2)) # jumping should be in first half
    bins = binMeasurement(measurement, jumpBinSize)
    jumpIndicies = list(bins.nlargest(numberOfJumps).sort_index().index)
    lowerBound, upperBound = jumpIndicies[0].left, jumpIndicies[numberOfJumps - 1].right
    return lowerBound - 100, upperBound + 100

def getJumps(measurement):
    windowIndicies = findJumpingWindow(measurement)
    window = measurement[windowIndicies[0]:windowIndicies[1]]
    jumpIntervals = list(binMeasurement(window, jumpBinSize).nlargest(numberOfJumps).sort_index().index)
    return [interval.left + np.argmax(window.loc[interval.left : interval.right].values) for interval in jumpIntervals]

In [ ]:
def alignSignals(dfX, dfY):
    dfxJumps = getJumps(dfX)
    dfyJumps = getJumps(dfY)
    
    offsets = [dfxJumps[iJump] - dfyJumps[iJump] for iJump in range(numberOfJumps)]
    return offsets[1] #sync on second jump

def alignAccelerationYWithRightFoot(measurements, location, axis):
    offset = alignSignals(
        measurements['foot-r']['accY_foot-r'], 
        measurements[location]['acc' + axis.upper() + '_' + location])
    measurements[location] = measurements[location].shift(offset, axis='index')

# Exercise detection

In [ ]:
zeroMovementThreshold = 0.75 # given in meters per second
minSubsequentZeroMovements = 200 

def getNextZeroBlock(series, startPosition):
    start = series[startPosition:][series == 0]
    if (len(start) == 0):
        raise ValueError
    start = start.index[0]
    iValue = start
    zeroCounter = 0
    while (iValue < len(series)):
        if (not series[iValue]):
            zeroCounter += 1
            iValue += 1
        elif (zeroCounter < minSubsequentZeroMovements):
            return getNextZeroBlock(series, iValue + 1)
        else:
            break
    return start, iValue - 1

def findAllNonZeroBlocks(series, startPosition):
    blocks = []
    start = series[startPosition:][series == 1].index[0]
    while (start < len(series)):
        try:
            zeroStart, zeroEnd = getNextZeroBlock(series, start)
            blocks.append((start, zeroStart - 1))
            start = zeroEnd + 1
        except ValueError:
            blocks.append((start, len(series) - 1))
            start = len(series)
    return blocks

def splitDataFrameIntoExercises(df, columnName):
    measurement = df[columnName]
    windowIndicies = findJumpingWindow(measurement)
    filteredByTH = measurement.abs().apply(lambda value: 1 if value > zeroMovementThreshold else 0)
    exerciseIntervals = findAllNonZeroBlocks(filteredByTH, windowIndicies[1])
    return list(map(lambda interval: df[interval[0] : interval[1]], exerciseIntervals))

## Combine loading, calibration, sync and exercise detection

In [ ]:
minExerciseLength = 300 # 10ms * minExerciseLength

def alignAll(measurements):
    alignAccelerationYWithRightFoot(measurements, 'hip-r', 'y')
    alignAccelerationYWithRightFoot(measurements, 'hip-l', 'y')
    alignAccelerationYWithRightFoot(measurements, 'foot-l', 'y')
    alignAccelerationYWithRightFoot(measurements, 'knee-l', 'y')
    alignAccelerationYWithRightFoot(measurements, 'knee-r', 'Y')
    
def calibrateAll(measurements):
    for location in measurements.values():
        for column in location.columns:
            calibrate(location[column])
            
def resetTimePointZero(mergedDf):
    firstIndex = max([mergedDf[column].first_valid_index() for column in mergedDf])
    lastIndex = min([mergedDf[column].last_valid_index() for column in mergedDf])
    return mergedDf[firstIndex:lastIndex]
        
def loadSyncedMeasurements(path):
    measurements = loadMeasurements(path)
    calibrateAll(measurements)
    alignAll(measurements)
    mergedDf = pd.DataFrame()
    for measurement in measurements.values():
        mergedDf = mergedDf.join(measurement, how='outer')
    mergedDf = resetTimePointZero(mergedDf).reset_index().drop('index', axis='columns')
    exercisesAndTurns = splitDataFrameIntoExercises(mergedDf, 'accY_foot-r')
    exercises = list(filter(lambda exerciseOrTurn: len(exerciseOrTurn) > minExerciseLength, exercisesAndTurns))
    data = [mergedDf] + exercises
    return data

In [ ]:
measurements = loadSyncedMeasurements(os.path.join('..', 'data', '24-06-19', 'dennis 1'))
print(len(measurements))

In [ ]:
measurements[0].plot(figsize=(20, 10))